In [2]:
import wandb
import pickle
import numpy as np
from glob import glob
import tensorflow as tf

from tensorflow.keras.layers import Input, BatchNormalization, Activation, Conv2D
from tensorflow.keras import Model

from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

tf.random.set_seed(999)
np.random.seed(999)


2023-04-18 08:01:27.741179: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-18 08:01:27.783724: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-18 08:01:27.784871: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 08:01:28.660235: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
# GLOBAL PARAMETERS
# Comment them all
WINDOW_LENGTH = 256
# Some comments why i need this variable
SAMPLE_RATE = 16000
# Why i need this variable
N_SEGMENTS = 8
# Why i need this variable
N_CLEAN_SEGMENTS = 1
# Why i need this variable
N_FFT = WINDOW_LENGTH
# Why i need this variable
N_FEATURES = N_FFT//2 + 1
# Why i need this variable
OVERLAP = round(0.25 * WINDOW_LENGTH)  # overlap of 75%

In [ ]:
# LOAD PREPARED DATA
DATA_PATH = '/run/media/svyatoslav/Files/ml_data_project'

x_train = pickle.load(open(DATA_PATH + '/X_training_4.pkl', 'rb'))
y_train = pickle.load(open(DATA_PATH + '/Y_training_4.pkl', 'rb'))

x_test = pickle.load(open(DATA_PATH + '/X_test_1.pkl', 'rb'))
y_test = pickle.load(open(DATA_PATH + '/Y_test_1.pkl', 'rb'))

x_val = pickle.load(open(DATA_PATH + '/X_val_1.pkl', 'rb'))
y_val = pickle.load(open(DATA_PATH + '/Y_val_1.pkl', 'rb'))

In [3]:
# Start a run, tracking hyperparameters
run = wandb.init(
    # set the wandb project where this run will be logged
    project="speech_hyper",

    # track hyperparameters and run metadata with wandb.config
    config={
        "activation": "relu",
        "loss": "mse",
        "metric": "rmse",
        "optimizer": "adam",
        "epoch": 32,
        "batch_size" : 2048,
        "learning_rate" : 0.0005
    }
)

config = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sams3pi01. Use `wandb login --relogin` to force relogin


In [8]:
# BUILD A MODEL
l2_strength = 0.0

# to implement skip connections we need to use functional API
inputs = Input(shape=[N_FEATURES, N_SEGMENTS, 1])
model = inputs
# -----
model = tf.keras.layers.ZeroPadding2D(((4, 4), (0, 0)))(model)
model = Conv2D(filters=18, kernel_size=[9, 8], strides=[1, 1], padding='valid', use_bias=False,
               kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(model)
model = Activation(config.activation)(model)
model = BatchNormalization()(model)

skip0 = Conv2D(filters=30, kernel_size=[5, 1], strides=[1, 1], padding='same', use_bias=False,
               kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(model)
model = Activation(config.activation)(skip0)
model = BatchNormalization()(model)

model = Conv2D(filters=8, kernel_size=[9, 1], strides=[1, 1], padding='same', use_bias=False,
               kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(model)
model = Activation(config.activation)(model)
model = BatchNormalization()(model)

# -----
model = Conv2D(filters=18, kernel_size=[9, 1], strides=[1, 1], padding='same', use_bias=False,
               kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(model)
model = Activation(config.activation)(model)
model = BatchNormalization()(model)

skip1 = Conv2D(filters=30, kernel_size=[5, 1], strides=[1, 1], padding='same', use_bias=False,
               kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(model)
model = Activation(config.activation)(skip1)
model = BatchNormalization()(model)

model = Conv2D(filters=8, kernel_size=[9, 1], strides=[1, 1], padding='same', use_bias=False,
               kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(model)
model = Activation(config.activation)(model)
model = BatchNormalization()(model)

# ----
model = Conv2D(filters=18, kernel_size=[9, 1], strides=[1, 1], padding='same', use_bias=False,
               kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(model)
model = Activation(config.activation)(model)
model = BatchNormalization()(model)

model = Conv2D(filters=30, kernel_size=[5, 1], strides=[1, 1], padding='same', use_bias=False,
               kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(model)
model = Activation(config.activation)(model)
model = BatchNormalization()(model)

model = Conv2D(filters=8, kernel_size=[9, 1], strides=[1, 1], padding='same', use_bias=False,
               kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(model)
model = Activation(config.activation)(model)
model = BatchNormalization()(model)

# ----
model = Conv2D(filters=18, kernel_size=[9, 1], strides=[1, 1], padding='same', use_bias=False,
               kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(model)
model = Activation(config.activation)(model)
model = BatchNormalization()(model)

model = Conv2D(filters=30, kernel_size=[5, 1], strides=[1, 1], padding='same', use_bias=False,
               kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(model)
model = model + skip1
model = Activation(config.activation)(model)
model = BatchNormalization()(model)

model = Conv2D(filters=8, kernel_size=[9, 1], strides=[1, 1], padding='same', use_bias=False,
               kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(model)
model = Activation(config.activation)(model)
model = BatchNormalization()(model)

# ----
model = Conv2D(filters=18, kernel_size=[9, 1], strides=[1, 1], padding='same', use_bias=False,
               kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(model)
model = Activation(config.activation)(model)
model = BatchNormalization()(model)

model = Conv2D(filters=30, kernel_size=[5, 1], strides=[1, 1], padding='same', use_bias=False,
               kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(model)
model = model + skip0
model = Activation(config.activation)(model)
model = BatchNormalization()(model)

model = Conv2D(filters=8, kernel_size=[9, 1], strides=[1, 1], padding='same', use_bias=False,
               kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(model)
model = Activation(config.activation)(model)
model = BatchNormalization()(model)

# ----
model = tf.keras.layers.SpatialDropout2D(0.2)(model)
model = Conv2D(filters=1, kernel_size=[129, 1], strides=[
               1, 1], padding='same')(model)

model = Model(inputs=inputs, outputs=model)

# optimizer = tf.keras.optimizers.Adam(learning_rate=config.learning_rate, epsilon = 1e-8)
# model.compile(optimizer=optimizer, loss=config.loss, metrics=[
#               tf.keras.metrics.RootMeanSquaredError(config.metric)])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 129, 8, 1)]  0           []                               
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 137, 8, 1)   0           ['input_3[0][0]']                
 D)                                                                                               
                                                                                                  
 conv2d_32 (Conv2D)             (None, 129, 1, 18)   1296        ['zero_padding2d_2[0][0]']       
                                                                                                  
 activation_30 (Activation)     (None, 129, 1, 18)   0           ['conv2d_32[0][0]']        

In [ ]:
# get last artefact from w&b
# artifact = run.use_artifact('sams3pi01/speech_denoising/run_3zig82av_model:v0', type='model')
# artifact_dir = artifact.download()

In [ ]:
import keras
keras.backend.clear_session()

optimizer = tf.keras.optimizers.Adam(learning_rate=config.learning_rate, epsilon = 1e-8)

model = keras.models.load_model('models/')
model.compile(optimizer=optimizer, loss=config.loss, metrics=[
              tf.keras.metrics.RootMeanSquaredError(config.metric)])

In [ ]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4, mode="min", restore_best_weights=True)

history = model.fit(
    x=x_train,
    y=y_train,
    epochs=config.epoch,

    batch_size = config.batch_size,
    validation_data=(x_val, y_val),
    callbacks=[
        WandbMetricsLogger(log_freq=5),
        WandbModelCheckpoint("models"),
        early_stopping_callback
    ]
)

wandb.agent(sweep_id, function=train)
model.save('model_1.h5')


val_loss = model.evaluate(x_test, y_test)[0]
print(val_loss)

In [ ]:
# Visualize loss
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# Visualize mse
plt.plot(history.history['rmse'])
plt.plot(history.history['val_rmse'])
plt.title('model rmse value')
plt.ylabel('rmse')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()